In [22]:
from google_agent import Google_agent
from deep_research import Deep_research_engine
from pydantic_ai import Agent, RunContext
from pydantic_ai.common_tools.tavily import tavily_search_tool
from pydantic_ai.messages import ModelMessage
from pydantic_ai.models.google import GoogleModel
from pydantic_ai.providers.google import GoogleProvider
from dotenv import load_dotenv
from dataclasses import dataclass
from datetime import datetime
from pydantic import Field

from langchain_google_genai import ChatGoogleGenerativeAI
import nest_asyncio
nest_asyncio.apply()
from langchain_openai import ChatOpenAI

import os
from tavily import TavilyClient
load_dotenv()
google_api_key=os.getenv('google_api_key')
tavily_key=os.getenv('tavily_key')
pse=os.getenv('pse')
openai_api_key=os.getenv('openai_api_key')
tavily_client = TavilyClient(api_key=tavily_key)
composio_api_key=os.getenv('composio_api_key')
# configure logfire
import logfire
logfire.configure(token=os.getenv('logfire_token'))
logfire.instrument_pydantic_ai()


Logfire project URL: https://logfire-us.pydantic.dev/padioutristan/cortana


In [23]:

api_keys={
    'google_api_key':google_api_key,
    'tavily_key':tavily_key,
    'pse':pse,
    'openai_api_key':openai_api_key,
    'composio_key':composio_api_key
}


In [24]:
@dataclass
class Message_state:
    messages: list[ModelMessage]

@dataclass
class Api_keys:
    api_keys: dict

@dataclass
class Deps:
    deep_research_output: dict
    mail_inbox: dict
    google_agent_output: dict

In [25]:
class Cortana_agent:
    def __init__(self, api_keys:dict):
        """
        Args:
            
            api_keys (dict): The API keys to use as a dictionary
        """

        GEMINI_MODEL='gemini-2.0-flash'
        self.api_keys=Api_keys(api_keys=api_keys)
       
        # tools
        llms={'pydantic_llm':GoogleModel('gemini-2.5-flash-preview-05-20', provider=GoogleProvider(api_key=self.api_keys.api_keys['google_api_key'])),
              'langchain_llm':ChatGoogleGenerativeAI(google_api_key=self.api_keys.api_keys['google_api_key'], model=GEMINI_MODEL, temperature=0.3),
              'openai_llm':ChatOpenAI(api_key=self.api_keys.api_keys['openai_api_key'])}
        
        
        google_agent=Google_agent(llms,self.api_keys.api_keys)
        async def google_agent_tool(ctx:RunContext[Deps],query:str):
            """
            # Google Agent Interaction Function

            ## Purpose
            This function provides an interface to interact with a Google agent that can perform multiple Google-related tasks simultaneously.

            ## Capabilities
            The agent can:
            - Search for images
            - Manage user emails
            - Manage Google tasks
            - Manage Google Maps
            - get contact list
            - List available tools
            - Improve planning based on user feedback
            - Improve its query based on user feedback

            ## Parameters
            - `query` (str): A complete query string describing the desired Google agent actions
            - The query should include all necessary details for the requested operations
            - Multiple actions can be specified in a single query

            ## Returns
            - `str`: The agent's response to the query

            ## Important Notes
            - The agent can process multiple actions in a single query
            - User feedback can be provided to help improve the agent's planning and query
            - All Google-related operations should be included in the query string

            """

           
            res=google_agent.chat(query)
            ctx.deps.google_agent_output['node_messages']=res.node_messages
            if google_agent.state.mail_inbox:
                ctx.deps.mail_inbox=google_agent.state.mail_inbox
            ctx.deps.google_agent_output=google_agent.state
            try:
                return res.node_messages[-1]
            except:
                return res
        


        async def search_and_question_answering_tool(ctx: RunContext[Deps], query:str, route:str):
            """
            Use this tool to do a deep research on a topic, to gather detailed informations and data, answer_questions from the deep research results or do a quick research if the answer is not related to the deep research.
            Args:
                query (str): The query related to the search_and_question_answering_tool and its capabilities
                route (str): The route, either deep_research or answer_question, or quick_research
                

            Returns:
                str: The response from the search_and_question_answering_tool
            """
            deep_research_engine=Deep_research_engine(llms['pydantic_llm'],self.api_keys.api_keys)
            @dataclass
            class Route:
                answer: str = Field(default_factory=None,description="the answer to the question if the question is related to the deep research")
                route: str = Field(description="the route, either deep_research or answer_question, or quick_research")
            agent=Agent(llms['pydantic_llm'], output_type=Route, instructions="you are a router/question answering agent, you are given a query and you need to decide what to do based on the information provided")
            response= agent.run_sync(f"based on the query: {query}, and the information provided: {ctx.deps.deep_research_output if ctx.deps.deep_research_output else ''} either answer the question or if the answer is not related to the information provided or need more information return 'quick_research' or 'deep_research'")
            route=response.output.route
            if route=='deep_research':
                response=deep_research_engine.chat(query)
                ctx.deps.deep_research_output=response
                return response
            elif route=='answer_question':
                return response.output.answer
            elif route=='quick_research':
                quick_research_agent=Agent(llms['pydantic_llm'], tools=[tavily_search_tool(self.api_keys.api_keys['tavily_key'])], instructions="do a websearch based on the query")
                result= quick_research_agent.run_sync(query)
                return result.output

        async def get_current_time_tool():
            """
            Use this tool to get the current time.
            Returns:
                str: The current time in a formatted string
            """
        
            return f"The current time is {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}"
                

        @dataclass
        class Cortana_output:
            ui_version: str= Field(description='a markdown format version of the answer for displays if necessary')
            voice_version: str = Field(description='a conversationnal version of the answer for text to voice')
        self.agent=Agent(llms['pydantic_llm'], output_type=Cortana_output, tools=[google_agent_tool, search_and_question_answering_tool, get_current_time_tool], system_prompt="you are Cortana, a helpful assistant that can help with a wide range of tasks,\
                          you can use the tools provided to you if necessary to help the user with their queries, ask how you can help the user")
        self.memory=Message_state(messages=[])
        self.deps=Deps(deep_research_output={}, google_agent_output={},mail_inbox={})
    
    def chat(self, query:any):
        """
        # Chat Function Documentation

        This function enables interaction with the user through various types of input.

        ## Parameters

        - `query`: The input to process. Can be one of the following types:
        - String: Direct text input passed to the agent
        - Binary content: Special format for media files (see below)

        ## Binary Content Types

        The function supports different types of media through `BinaryContent` objects:

        ### Audio
        ```python
        cortana_agent.chat([
            'optional string message',
            BinaryContent(data=audio, media_type='audio/wav')
        ])
        ```

        ### PDF Files
        ```python
        cortana_agent.chat([
            'optional string message',
            BinaryContent(data=pdf_path.read_bytes(), media_type='application/pdf')
        ])
        ```

        ### Images
        ```python
        cortana_agent.chat([
            'optional string message',
            BinaryContent(data=image_response.content, media_type='image/png')
        ])
        ```

        ## Returns

        - `Cortana_output`: as a pydantic object, the ui_version and voice_version are the two fields of the object

        ## Extra Notes
        The deps and message_history of cortana can be accessed using the following code:
        ```python
        cortana_agent.deps
        cortana_agent.memory.messages
        ```
        """

        result=self.agent.run_sync(query, deps=self.deps, message_history=self.memory.messages)
        self.memory.messages=result.all_messages()
        return result.output
    def reset(self):
        """
        Resets the Cortana agent to its initial state.

        Returns:
            str: A confirmation message indicating that the agent has been reset.
        """
        self.memory.messages=[]
        self.deps=Deps(deep_research_output={}, google_agent_output={},mail_inbox={})
        return f'Cortana has been reset'


In [26]:
cortana_agent=Cortana_agent(api_keys)

c:\Users\trist\OneDrive\Desktop\important\ai_portfolio\Cortana\.venv\Lib\site-packages\composio\client\collections.py:1183: UserWarning: Using all actions of an app is not recommended for production.Learn more: https://docs.composio.dev/patterns/tools/use-tools/use-specific-actions


  warnings.warn(


In [27]:
res=cortana_agent.chat('list tools')
print(res.ui_version)

22:20:27.230 agent run
22:20:27.243   chat gemini-2.5-flash-preview-05-20
22:20:28.042   running 1 tool
22:20:28.043     running tool: google_agent_tool
22:20:28.047       run graph None
22:20:28.048         run node planner_node
22:20:28.052           plan_agent run
22:20:28.053             chat gemini-2.5-flash-preview-05-20
22:20:28.851         run node agent_node
22:20:28.857           agent run
22:20:28.858             chat gemini-2.5-flash-preview-05-20
22:20:30.628         run node list_tools_node
22:20:30.632   chat gemini-2.5-flash-preview-05-20
I can help with various tasks using the following tools:

*   **Google Agent Tools:**
    *   **Mail Manager:** Send emails, fetch emails, list drafts, delete messages/drafts, reply to threads, get contacts, and search people.
    *   **Maps Manager:** Get directions, compute routes, and search for places (nearby or text search).
    *   **Tasks Manager:** Create/delete/update task lists, insert/delete/get/list/patch tasks, and clear c

In [15]:
print(res.voice_version)

Here are your tasks: You have a task titled Massage gun, due on May 28, 2025. You have a task titled Find information about the dust detection experiment, due on January 28, 2024. It has notes. You have a task titled Find information about the passive seismic experiment, due on December 31, 2023. It has notes. You have a task titled Find information about the laser ranging retroreflector experiment, due on December 31, 2023. It has notes. You have a task titled Find information about the solar wind composition experiment, due on October 27, 2023. It has notes. You have a task titled Find information about the lunar soil mechanics experiment, due on December 31, 2023. It has notes. And finally, you have a task titled List experiments conducted during the moon landing, due on December 31, 2023. It also has notes.


In [11]:
cortana_agent.deps.google_agent_output

State(node_messages=[{'tasks_manager': {'data': {'response_data': {'etag': '"Eyxr844Jbec"', 'items': [{'etag': '"3OPBIJSFMs8"', 'id': 'MDMyNDkzMjIzMzQ5ODI4MTU2MjY6MDow', 'kind': 'tasks#taskList', 'selfLink': 'https://www.googleapis.com/tasks/v1/users/@me/lists/MDMyNDkzMjIzMzQ5ODI4MTU2MjY6MDow', 'title': 'My Tasks', 'updated': '2025-05-28T04:19:44.162Z'}], 'kind': 'tasks#taskLists'}}, 'error': None, 'successfull': True, 'successful': True, 'logId': 'log_pZ5heDwm4DCr'}}, {'tasks_manager': {'data': {'nextPageToken': None, 'tasks': [{'completed': None, 'deleted': None, 'due': '2025-05-28T00:00:00.000Z', 'etag': '"c_kqt2t3vuc"', 'hidden': None, 'id': 'SWVpSW9sT2lTd0VOMnFTdw', 'kind': 'tasks#task', 'links': [], 'notes': None, 'parent': None, 'position': '00000000000000000000', 'selfLink': 'https://www.googleapis.com/tasks/v1/lists/MDMyNDkzMjIzMzQ5ODI4MTU2MjY6MDow/tasks/SWVpSW9sT2lTd0VOMnFTdw', 'status': 'needsAction', 'title': 'Massage gun', 'updated': '2025-05-28T04:19:44.162Z', 'webViewLin

In [13]:
cortana_agent.memory.messages

[ModelRequest(parts=[SystemPromptPart(content='you are Cortana, a helpful assistant that can help with a wide range of tasks,                          you can use the tools provided to you to help the user with their queries, ask how you can help the user', timestamp=datetime.datetime(2025, 5, 28, 17, 49, 10, 540841, tzinfo=datetime.timezone.utc)), UserPromptPart(content='list tasks in My Tasks tasklist', timestamp=datetime.datetime(2025, 5, 28, 17, 49, 10, 540848, tzinfo=datetime.timezone.utc))]),
 ModelResponse(parts=[ToolCallPart(tool_name='google_agent_tool', args={'query': 'list tasks in My Tasks tasklist'}, tool_call_id='pyd_ai_669cf3684cac40e8b0ed7a16535d5e00')], usage=Usage(requests=1, request_tokens=491, response_tokens=23, total_tokens=585, details={'thoughts_token_count': 71}), model_name='models/gemini-2.5-flash-preview-05-20', timestamp=datetime.datetime(2025, 5, 28, 17, 49, 11, 560350, tzinfo=datetime.timezone.utc)),
 ModelRequest(parts=[ToolReturnPart(tool_name='google_a